In [10]:
import pandas as pd
import json

df = pd.read_json("anatole_data_train.json")
df_train=df.sample(frac=0.95,random_state=0)
df_eval=df.drop(df_train.index)

df_formatted = [
    {
        "messages": [
            {"role": "user", "content": row["instruction"]},
            {"role": "assistant", "content": row["output"]},
        ]
    }
    for index, row in df_train.iterrows()
]

df_formatted2 = [
    {
        "messages": [
            {"role": "user", "content": row["instruction"]},
            {"role": "assistant", "content": row["output"]},
        ]
    }
    for index, row in df_eval.iterrows()
]

with open("data_train.jsonl", "w") as f:
    for line in df_formatted:
        json.dump(line, f)
        f.write("\n")
with open("data_eval.jsonl", "w") as f:
    for line in df_formatted2:
        json.dump(line, f)
        f.write("\n")

In [29]:
len(df)

31077

In [13]:
import os
from mistralai.client import MistralClient
os.environ['MISTRAL_API_KEY']='ymjotzDIdJxKj611QnrKt70VEJsHtyGm'
api_key = os.environ.get("MISTRAL_API_KEY")
client = MistralClient(api_key=api_key)

with open("data_train.jsonl", "rb") as f:
    data_train = client.files.create(file=("data_train.jsonl", f))
with open("data_eval.jsonl", "rb") as f:
    data_eval = client.files.create(file=("data_eval.jsonl", f))

In [35]:
from mistralai.models.jobs import TrainingParameters

created_jobs = client.jobs.create(
    model="open-mistral-7b",
    training_files=[data_train.id],
    validation_files=[data_eval.id],
    hyperparameters=TrainingParameters(
        training_steps=1,
        learning_rate=0.0001,
        )
)
created_jobs

Job(id='2cad6ef8-34b8-4e20-9431-db9ccccb0f1d', hyperparameters=TrainingParameters(training_steps=180, learning_rate=0.0001), fine_tuned_model=None, model='open-mistral-7b', status='QUEUED', job_type='FT', created_at=1719433586, modified_at=1719433586, training_files=['5cd53dbf-4f32-45e6-9b15-0c69b05b2cc6'], validation_files=['ff6b09ef-e7ba-4c1f-8a25-a35681ef8c93'], object='job', integrations=[])

In [34]:
from mistralai.models.jobs import TrainingParameters


dry_run_job = client.jobs.create(
    model="open-mistral-7b",
    training_files=[data_train.id],
    validation_files=[data_eval.id],
    hyperparameters=TrainingParameters(
        training_steps=180,
        learning_rate=0.0002,
    ),
    dry_run=True,
)
print(dry_run_job)


object='job.metadata' training_steps=180 train_tokens_per_step=131072 data_tokens=4455124 train_tokens=23592960 epochs=5.2957 expected_duration_seconds=1080


In [48]:
retrieved_jobs = client.jobs.retrieve(created_jobs.id)


In [49]:
print(retrieved_jobs)

id='2cad6ef8-34b8-4e20-9431-db9ccccb0f1d' hyperparameters=TrainingParameters(training_steps=180, learning_rate=0.0001) fine_tuned_model='ft:open-mistral-7b:c6d4dfa8:20240626:2cad6ef8' model='open-mistral-7b' status='SUCCESS' job_type='FT' created_at=1719433586 modified_at=1719434336 training_files=['5cd53dbf-4f32-45e6-9b15-0c69b05b2cc6'] validation_files=['ff6b09ef-e7ba-4c1f-8a25-a35681ef8c93'] object='job' integrations=[] events=[Event(name='status-updated', data={'status': 'SUCCESS'}, created_at=1719434336), Event(name='status-updated', data={'status': 'RUNNING'}, created_at=1719433587), Event(name='status-updated', data={'status': 'QUEUED'}, created_at=1719433586)] checkpoints=[Checkpoint(metrics=Metric(train_loss=0.0853, valid_loss=0.094045, valid_mean_token_accuracy=1.067359), step_number=180, created_at=1719434303), Checkpoint(metrics=Metric(train_loss=0.091283, valid_loss=0.094337, valid_mean_token_accuracy=1.067575), step_number=144, created_at=1719434160), Checkpoint(metrics=M